# Fit $k_{ij}$ interactions parameter of Hexane and Ethanol


---
Let's call $\underline{\xi}$ the optimization parameters of a mixture. In order to optimize them, you need to provide experimental phase equilibria data. This can include VLE, LLE and VLLE data. The objective function used for each equilibria type are shown below:


### Vapor-Liquid Equilibria Data


$$ OF_{VLE}(\underline{\xi}) = w_y \sum_{j=1}^{Np} \left[ \sum_{i=1}^c (y_{i,j}^{cal} - y_{i,j}^{exp})^2 \right] +  w_P \sum_{j=1}^{Np} \left[ \frac{P_{j}^{cal} - P_{j}^{exp}}{P_{j}^{exp}} \right]^2$$

Where, $Np$ is the number of experimental data points, $y_i$ is the vapor molar fraction of the component $i$ and $P$ is the bubble pressure. The superscripts $cal$ and $exp$ refers to the computed and experimental values, respectively. Finally, $w_y$ is the weight for the vapor composition error and $w_P$ is the weight for the bubble pressure error.


### Liquid-Liquid Equilibria Data
$$ OF_{LLE}(\underline{\xi}) = w_x \sum_{j=1}^{Np} \sum_{i=1}^c  \left[x_{i,j} - x_{i,j}^{exp}\right]^2 +  w_w \sum_{j=1}^{Np} \sum_{i=1}^c \left[ w_{i,j} - w_{i,j}^{exp}) \right]^2 $$

Where, $Np$ is the number of experimental data points, $x_i$ and $w_i$ are the molar fraction of the component $i$ on the liquids phases. The superscripts $cal$ and $exp$ refers to the computed and experimental values, respectively. Finally, $w_x$ and $w_w$ are the weights for the liquid 1 ($x$) and liquid 2 ($w$) composition error.


### Vapor-Liquid-Liquid Equilibria Data
$$ OF_{VLLE}(\underline{\xi}) = w_x \sum_{j=1}^{Np}  \sum_{i=1}^c  \left[x_{i,j}^{cal} - x_{i,j}^{exp}\right]^2 + w_w \sum_{j=1}^{Np}  \sum_{i=1}^c  \left[w_{i,j}^{cal} - w_{i,j}^{exp}\right]^2 +  w_y \sum_{j=1}^{Np}  \sum_{i=1}^c  \left[y_{i,j}^{cal} - y_{i,j}^{exp}\right]^2 + w_P \sum_{j=1}^{Np}  \left[ \frac{P_{j}^{cal}}{P_{j}^{exp}} - 1\right]^2 $$

Where, $Np$ is the number of experimental data points, $y_i$, $x_i$ and $w_i$ are the molar fraction of the component $i$ on the vapor and liquids phases, respectively. The superscripts $cal$ and $exp$ refers to the computed and experimental values, respectively. Finally, $w_x$ and $w_w$ are the weights for the liquid 1 ($x$) and liquid 2 ($w$) composition error, $w_y$ is the weight for vapor composition error and $w_P$ is weight for three phase equilibria pressure error.

If there is data for more than one equilibria type, the errors can be added accordinly. So the objective funcion becomes:

$$ OF(\underline{\xi}) =OF_{ELV}(\underline{\xi}) + OF_{ELL}(\underline{\xi}) + OF_{ELLV}(\underline{\xi})$$


---
This notebook has te purpose of showing how to optimize the $k_{ij}$ for a mixture in SGTPy. This parameters corrects the cross-interaction energy used for Mie potential interactions:

$$ \epsilon_{ij} = (1-k_{ij}) \frac{\sqrt{\sigma_i^3 \sigma_j^3}}{\sigma_{ij}^3} \sqrt{\epsilon_i \epsilon_j}$$ 


First, it's needed to import the necessary modules:

In [1]:
import numpy as np

from SGTPy import component, mixture, saftvrmie
from SGTPy.fit import fit_kij

Now that the functions are available it is necessary to create the mixture.

In [2]:
ethanol = component('ethanol2C', ms = 1.7728, sigma = 3.5592 , eps = 224.50,
                    lambda_r = 11.319, lambda_a = 6., eAB = 3018.05, rcAB = 0.3547,
                    rdAB = 0.4, sites = [1,0,1], cii= 5.3141080872882285e-20)

hexane = component('hexane', ms = 1.96720036, sigma = 4.54762477, eps = 377.60127994, 
                   lambda_r = 18.41193194, cii = 3.581510586936205e-19)

mix = mixture(hexane, ethanol)

Now the experimental equilibria data is read and a tuple is created. It includes the experimental liquid composition, vapor composition, equilibrium temperature and pressure. This is done with ```datavle = (Xexp, Yexp, Texp, Pexp)```


In [3]:
# Experimental data obtained from Sinor, Weber, J. Chem. Eng. Data, vol. 5, no. 3, pp. 243–247, 1960.

# Experimental temperature saturation in K
Texp = np.array([351.45, 349.15, 346.35, 340.55, 339.05, 334.95, 332.55, 331.85,
       331.5 , 331.25, 331.15, 331.4 , 331.6 , 332.3 , 333.35, 336.65,
       339.85, 341.85])

# Experimental pressure in Pa
Pexp = np.array([101330., 101330., 101330., 101330., 101330., 101330., 101330.,
       101330., 101330., 101330., 101330., 101330., 101330., 101330.,
       101330., 101330., 101330., 101330.])

# Experimental liquid composition
Xexp = np.array([[0.   , 0.01 , 0.02 , 0.06 , 0.08 , 0.152, 0.245, 0.333, 0.452,
        0.588, 0.67 , 0.725, 0.765, 0.898, 0.955, 0.99 , 0.994, 1.   ],
       [1.   , 0.99 , 0.98 , 0.94 , 0.92 , 0.848, 0.755, 0.667, 0.548,
        0.412, 0.33 , 0.275, 0.235, 0.102, 0.045, 0.01 , 0.006, 0.   ]])

# Experimental vapor composition
Yexp = np.array([[0.   , 0.095, 0.193, 0.365, 0.42 , 0.532, 0.605, 0.63 , 0.64 ,
        0.65 , 0.66 , 0.67 , 0.675, 0.71 , 0.745, 0.84 , 0.935, 1.   ],
       [1.   , 0.905, 0.807, 0.635, 0.58 , 0.468, 0.395, 0.37 , 0.36 ,
        0.35 , 0.34 , 0.33 , 0.325, 0.29 , 0.255, 0.16 , 0.065, 0.   ]])

datavle = (Xexp, Yexp, Texp, Pexp)

The function ```fit_kij``` optimize the $k_{ij}$. The function requires bounds for the parameter, as well as the mixture object and the equilibria data. Optionally, the ```minimize_options``` option allows modifying the minimizer default parameters.

In [4]:
# bounds fors kij
kij_bounds = (-0.01, 0.01)
fit_kij(kij_bounds, mix, datavle = datavle)

     fun: 0.00047846435253596435
    nfev: 25
     nit: 21
 success: True
       x: 0.011818396009358412

If the mixture exhibits other equilibria types you can supply this experimental data to the ``datalle`` or ``datavlle`` parameters.

- ``datalle``: (Xexp, Wexp, Texp, Pexp)
- ``datavlle``: (Xexp, Wexp, Yexp, Texp, Pexp)

You can specify the weights for each objetive function through the following parameters:

- ``weights_vle``: list or array_like, weights for the VLE objective function.
        - weights_vle[0] = weight for Y composition error, default to 1.
        - weights_vle[1] = weight for bubble pressure error, default to 1.
- ``weights_lle``: list or array_like, weights for the LLE objective function.
        - weights_lle[0] = weight for X (liquid 1) composition error, default to 1.
        - weights_lle[1] = weight for W (liquid 2) composition error, default to 1.
- ``weights_vlle``: list or array_like, weights for the VLLE objective function.
        - weights_vlle[0] = weight for X (liquid 1) composition error, default to 1.
        - weights_vlle[1] = weight for W (liquid 2) composition error, default to 1.
        - weights_vlle[2] = weight for Y (vapor) composition error, default to 1.
        - weights_vlle[3] = weight for equilibrium pressure error, default to 1.

Additionally, you can set options to the SciPy's ``minimize_scalar`` function using the ``minimize_options`` parameter.

For more information just run:
```fit_kij?```